## Making our data generators
We will use ImageDataGenerator to implement data augmentation.

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   validation_split = 0.2)
train_data = train_datagen.flow_from_directory('./prepared_data/train_data/', target_size = (80,80), batch_size = 8, class_mode = 'binary', subset = 'training')
validation_data = train_datagen.flow_from_directory('./prepared_data/train_data/', target_size = (80,80), batch_size = 8, class_mode = 'binary', subset = 'validation')

Found 51200 images belonging to 2 classes.
Found 12800 images belonging to 2 classes.


In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory('./prepared_data/test_data', target_size = (80,80), batch_size = 8, class_mode = 'binary')

Found 20898 images belonging to 2 classes.


## Making our model
We will use the concept of Transfer Learning and use the Inception model as the base model to make our CNN.

In [26]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [27]:
# we will make our own dense layers so we do not need the final dense layers of Inception
base_model = InceptionV3(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (80,80,3)))
head_model = base_model.output

In [28]:
# we don't need to train layers from InceptionV3
for layer in base_model.layers:
    layer.trainable=False

In [29]:
# making the head model
head_model=Flatten()(head_model) # flatten the incoming image data to a 1d vector
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.2)(head_model) # avoids overfitting
head_model=Dense(1,activation='sigmoid')(head_model) # gives the class

model = Model(inputs = base_model.input, outputs = head_model)

## Training our head model

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint('./model checkpoint/model.keras', monitor = 'val_loss', save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, restore_best_weights = True)
learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience = 3)
callbacks = [checkpoint, earlystop, learning_rate]

In [31]:
batch_size = 80
print(train_data.samples // batch_size * 10)
print(train_data.samples)
print(validation_data.samples // batch_size * 10)
print(validation_data.samples)

6400
51200
1600
12800


In [32]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(train_data,
          steps_per_epoch = train_data.samples // batch_size,
          validation_data = validation_data,
          validation_steps = validation_data.samples // batch_size,
          callbacks = callbacks, 
          epochs = 10)

Epoch 1/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 99s 131ms/step - accuracy: 0.8757 - loss: 0.3038 - val_accuracy: 0.8711 - val_loss: 0.2865 - learning_rate: 0.0010
Epoch 2/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 66s 104ms/step - accuracy: 0.9230 - loss: 0.1975 - val_accuracy: 0.8484 - val_loss: 0.3281 - learning_rate: 0.0010
Epoch 3/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 51s 79ms/step - accuracy: 0.9212 - loss: 0.1927 - val_accuracy: 0.8797 - val_loss: 0.2795 - learning_rate: 0.0010
Epoch 4/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 38s 60ms/step - accuracy: 0.9289 - loss: 0.1835 - val_accuracy: 0.9164 - val_loss: 0.2642 - learning_rate: 0.0010
Epoch 5/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 36s 56ms/step - accuracy: 0.9285 - loss: 0.1801 - val_accuracy: 0.8813 - val_loss: 0.2651 - learning_rate: 0.0010
Epoch 6/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 36s 57ms/step - accuracy: 0.9329 - loss: 0.1701 - val_accuracy: 0.8875 - val_loss: 0.2660 - learning_rate: 0.0010
Epoch 7/10
640/640 ━━━━━━━━━━━━━━━━━━━━ 34s 54ms/step - accuracy: 0.9286 -

## Evaluating our model

In [34]:
loss_train, acc_train = model.evaluate(train_data)
print(acc_train)
print(loss_train)

6400/6400 ━━━━━━━━━━━━━━━━━━━━ 275s 43ms/step - accuracy: 0.9343 - loss: 0.1642
0.9344335794448853
0.16515152156352997


In [35]:
loss_val, acc_val = model.evaluate(validation_data)
print(acc_val)
print(loss_val)

1600/1600 ━━━━━━━━━━━━━━━━━━━━ 69s 43ms/step - accuracy: 0.9066 - loss: 0.2255
0.8993750214576721
0.2423509806394577


In [33]:
loss_test, acc_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

2613/2613 ━━━━━━━━━━━━━━━━━━━━ 107s 41ms/step - accuracy: 0.9443 - loss: 0.1476
0.9432960152626038
0.14809730648994446
